### Введение

Метод главных компонент (principal component analysis, PCA) — это один из методов обучения без учителя, который позволяет сформировать новые признаки, являющиеся линейными комбинациями старых. При этом новые признаки строятся так, чтобы сохранить как можно больше дисперсии в данных. Иными словами, метод главных компонент понижает размерность данных оптимальным с точки зрения сохранения дисперсии способом.

Основным параметром метода главных компонент является количество новых признаков. Как и в большинстве методов машинного обучения, нет четких рекомендаций по поводу выбора значения этого параметров. Один из подходов — выбирать минимальное число компонент, при котором объясняется не менее определенной доли дисперсии (это означает, что в выборке сохраняется данная доля от исходной дисперсии).

В этом задании понадобится измерять схожесть двух наборов величин. Если имеется набор пар измерений (например, одна пара — предсказания двух классификаторов для одного и того же объекта), то охарактеризовать их зависимость друг от друга можно с помощью корреляции Пирсона. Она принимает значения от -1 до 1 и показывает, насколько данные величины линейно зависимы. Если корреляция равна -1 или 1, то величины линейно выражаются друг через друга. Если она равна нулю, то линейная зависимость между величинами отсутствует.


### Данные

В этом задании мы будем работать с данными о стоимостях акций 30 крупнейших компаний США. На основе этих данных можно оценить состояние экономики, например, с помощью индекса Доу-Джонса. Со временем состав компаний, по которым строится индекс, меняется. Для набора данных был взят период с 23.09.2013 по 18.03.2015, в котором набор компаний был фиксирован (подробнее см. [Dow Jones Industrial Average](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average) и [История состава компаний, входящих в индекс Dow Jones](https://en.wikipedia.org/wiki/Historical_components_of_the_Dow_Jones_Industrial_Average)). 

Одним из существенных недостатков индекса Доу-Джонса является способ его вычисления — при подсчёте индекса цены входящих в него акций складываются, а потом делятся на поправочный коэффициент. В результате, даже если одна компания заметно меньше по капитализации, чем другая, но стоимость одной её акции выше, то она сильнее влияет на индекс. Даже большое процентное изменение цены относительно дешёвой акции может быть нивелировано незначительным в процентном отношении изменением цены более дорогой акции.


### Реализация в sklearn

Метод главных компонент реализован в пакете `scikit-learn` в модуле `decomposition` в классе [`PCA`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). Основным параметром является количество компонент (`n_components`). Для обученного преобразования этот класс позволяет вычислять различные характеристики. Например, поле `explained_variance_ratio_` содержит процент дисперсии, который объясняет каждая компонента. Поле `components_` содержит информацию о том, какой вклад вносят признаки в компоненты. Чтобы применить обученное преобразование к данным, можно воспользоваться методом [`transform`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.transform). 

Для нахождения коэффициента корреляции Пирсона можно воспользоваться функцией [`corrcoef`](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.corrcoef.html) из пакета `numpy`.


#### Загрузите данные close_prices.csv. 
В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый день периода. 

In [12]:
import pandas as pd

df = pd.read_csv('close_prices.csv')
df.shape

(374, 31)

In [13]:
df.head()

,date,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


#### На загруженных данных обучите преобразование PCA с числом компоненты равным 10. 
Скольких компонент хватит, чтобы объяснить 90% дисперсии?

In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
pca.fit(df.iloc[:,1:])

PCA(copy=True, n_components=10, whiten=False)

In [10]:
pca.explained_variance_ratio_, sum(pca.explained_variance_ratio_[:4])

(array([ 0.73897118,  0.11007169,  0.04995088,  0.0287492 ,  0.02215448,
         0.01931577,  0.00674853,  0.00614091,  0.00320594,  0.00305611]),
 0.92774295378364047)

#### Примените построенное преобразование к исходным данным и возьмите значения первой компоненты. 

In [19]:
# Напишите свой код
X = pca.transform(df.iloc[:,1:])
df2 = pd.DataFrame(X)
df2.head()

,0,1,2,3,4,5,6,7,8,9
0,-50.902404,-17.631677,-7.736021,2.596010,2.377569,4.244055,-12.883088,3.556570,-5.821974,-1.726040
1,-52.846909,-19.146907,-7.272546,4.618575,0.102182,3.935519,-11.062930,3.432599,-5.633181,-2.012232
2,-54.614439,-19.927358,-8.109958,4.920347,-0.421454,4.555335,-10.119224,4.175203,-5.388287,-2.752699
3,-52.600566,-19.941571,-6.948113,4.806243,0.319779,5.065505,-9.849605,3.266579,-5.974574,-1.922746
4,-52.370123,-22.289064,-8.410868,3.833372,-2.464882,4.359897,-7.988691,3.452709,-5.745204,-1.875131


In [21]:
df2[0]

0     -50.902404
1     -52.846909
2     -54.614439
3     -52.600566
4     -52.370123
5     -54.653412
6     -52.812575
7     -53.651146
8     -56.692727
9     -54.402655
10    -58.331645
11    -61.716265
12    -61.319400
13    -53.792524
14    -50.892634
15    -49.637408
16    -52.387615
17    -47.478543
18    -43.560335
19    -42.559280
20    -42.450271
21    -42.271248
22    -43.252728
23    -40.059345
24    -38.497263
25    -39.064923
26    -38.393044
27    -38.400558
28    -42.386170
29    -39.830574
         ...    
344    67.607802
345    74.206256
346    71.305000
347    68.041973
348    70.268799
349    71.435269
350    76.342232
351    74.859336
352    76.322836
353    75.621858
354    75.403574
355    80.946849
356    82.097270
357    83.045140
358    83.048730
359    84.251636
360    81.085307
361    88.953955
362    85.514365
363    83.441017
364    84.579747
365    77.973512
366    81.468090
367    72.450565
368    72.456036
369    81.290980
370    77.903534
371    83.7981

#### Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv. 
Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса? 

In [22]:
dj_df = pd.read_csv('djia_index.csv')
dj_df.head()

,date,^DJI
0,2013-09-23,15401.379883
1,2013-09-24,15334.589844
2,2013-09-25,15273.259766
3,2013-09-26,15328.299805
4,2013-09-27,15258.240234


In [24]:
import numpy as np
np.corrcoef(df2[0], dj_df['^DJI'])[0][1]

0.90965221930502393

#### Какая компания имеет наибольший вес в первой компоненте? 
Укажите ее название с большой буквы.

In [25]:
pca.components_[0]

array([  1.61383840e-02,   1.20644923e-01,  -5.16612711e-02,
         5.04842369e-02,  -1.25859933e-01,   1.14089567e-01,
         2.33906290e-01,  -6.20513749e-03,   2.51227032e-01,
         2.88996029e-01,  -2.64998795e-01,   9.31320168e-02,
         9.13948403e-02,   4.69879340e-02,   2.90549417e-02,
        -2.61068828e-02,   3.29615584e-01,   7.13897133e-02,
         7.62295699e-02,   2.11888868e-01,   2.30922941e-02,
         7.77316954e-02,  -7.20594590e-03,   1.89479745e-01,
         3.21564017e-01,   5.36834873e-02,   5.79683946e-01,
         1.09122230e-04,   8.71614334e-02,  -4.29421420e-02])

In [28]:
tup_comp = list(zip(df.columns[1:], pca.components_[0]))
tup_comp

[('AXP', 0.016138383984734751),
 ('BA', 0.12064492322861042),
 ('CAT', -0.051661271080883253),
 ('CSCO', 0.050484236916854067),
 ('CVX', -0.12585993303584916),
 ('DD', 0.11408956671451209),
 ('DIS', 0.23390628951791737),
 ('GE', -0.0062051374900815642),
 ('GS', 0.25122703209089808),
 ('HD', 0.28899602916707423),
 ('IBM', -0.26499879525484737),
 ('INTC', 0.09313201682640565),
 ('JNJ', 0.091394840292063101),
 ('JPM', 0.046987934026593556),
 ('KO', 0.029054941715540447),
 ('MCD', -0.026106882759563028),
 ('MMM', 0.32961558381585271),
 ('MRK', 0.071389713288802586),
 ('MSFT', 0.076229569876511541),
 ('NKE', 0.21188886780132193),
 ('PFE', 0.023092294068853499),
 ('PG', 0.077731695391623057),
 ('T', -0.0072059458950082282),
 ('TRV', 0.18947974464619469),
 ('UNH', 0.32156401737619073),
 ('UTX', 0.053683487328683263),
 ('V', 0.57968394574736104),
 ('VZ', 0.00010912222981037261),
 ('WMT', 0.087161433382868164),
 ('XOM', -0.042942141992919708)]

In [34]:
sorted(tup_comp, key=lambda x: abs(x[1]), reverse=True)

[('V', 0.57968394574736104),
 ('MMM', 0.32961558381585271),
 ('UNH', 0.32156401737619073),
 ('HD', 0.28899602916707423),
 ('IBM', -0.26499879525484737),
 ('GS', 0.25122703209089808),
 ('DIS', 0.23390628951791737),
 ('NKE', 0.21188886780132193),
 ('TRV', 0.18947974464619469),
 ('CVX', -0.12585993303584916),
 ('BA', 0.12064492322861042),
 ('DD', 0.11408956671451209),
 ('INTC', 0.09313201682640565),
 ('JNJ', 0.091394840292063101),
 ('WMT', 0.087161433382868164),
 ('PG', 0.077731695391623057),
 ('MSFT', 0.076229569876511541),
 ('MRK', 0.071389713288802586),
 ('UTX', 0.053683487328683263),
 ('CAT', -0.051661271080883253),
 ('CSCO', 0.050484236916854067),
 ('JPM', 0.046987934026593556),
 ('XOM', -0.042942141992919708),
 ('KO', 0.029054941715540447),
 ('MCD', -0.026106882759563028),
 ('PFE', 0.023092294068853499),
 ('AXP', 0.016138383984734751),
 ('T', -0.0072059458950082282),
 ('GE', -0.0062051374900815642),
 ('VZ', 0.00010912222981037261)]